# Importing packages 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

# Importing Twitter API and Access

In [2]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

# Credentials for twitter

In [3]:
consumer_key = "nucwiQk9GtsJfWFSGQjXFicnq"
consumer_secret = "psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV"
access_token = "296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7"
access_token_secret = "FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU"

# Calling API and working on it 

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
# Lets provide the query which I need to search

query = "US election Joe Biden"

# Fetch tweets

Tweets = api.search(query, count=10, lang= 'en', exclude= 'retweets', tweet_mode= 'extended')

for tweet in Tweets:
    print(tweet)
    print("=================================================")

Status(_api=<tweepy.api.API object at 0x00000290CDC719D0>, _json={'created_at': 'Wed Jun 23 06:06:04 +0000 2021', 'id': 1407580697133453313, 'id_str': '1407580697133453313', 'full_text': 'TEXAS STATE BAR ASSOCIATION INVESTIGATING ABBOTT FOR ATTEMPTING TO REVERSE ELECTION \nDid bogus claims of fraud amount to professional misconduct? Dismissing the voters will is normally punishable with immediate expulsion. #sedition\nhttps://t.co/wsNEKiVkCg', 'truncated': False, 'display_text_range': [0, 255], 'entities': {'hashtags': [{'text': 'sedition', 'indices': [222, 231]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/wsNEKiVkCg', 'expanded_url': 'https://apnews.com/article/donald-trump-joe-biden-us-supreme-court-texas-election-2020-071cd81a1447f17bfd115edffc4ff456', 'display_url': 'apnews.com/article/donald…', 'indices': [232, 255]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="n

# Getting the Tweets and some of its related Attributes

In [6]:
df = pd.DataFrame(columns=[
    'Tweets', 'User', 'User_statuses_count',
    'user_followers', 'User_location', 'User_verified',
    'fav_count', 'rt_count', 'tweet_date'
])
df

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date


# Importing 1000 tweets related to given topic

In [7]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["US election Joe Biden"], file_name='my_tweets')  #Importing tweets

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,C. Anthony Null,41981,1479,"Corinth, MS 38834",False,0,7,2021-06-23 06:18:19
1,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,ABerman333,94023,1735,,False,0,7,2021-06-23 06:17:34
2,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,Mina A.S,16419,457,London,False,0,7,2021-06-23 06:16:44
3,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,TC Tunstall,22240,1506,Seattle,False,0,7,2021-06-23 06:10:18
4,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,Adrian German,19774,2446,"Bloomington, IN",False,0,7,2021-06-23 06:08:51


In [10]:
df.shape

(1000, 9)

In [11]:
from textblob import TextBlob  #importing textblob for sentiment analysis

In [12]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))  #cleaning tweets

In [15]:
df[['Tweets','clean_tweet']].head()

,Tweets,clean_tweet
0,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...
1,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...
2,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...
3,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...
4,RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...


In [16]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))   #analysis

In [17]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
1,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
2,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
3,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
4,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral


In [18]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [19]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
1,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
2,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
3,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral
4,RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...,Neutral


In [20]:
print(df.head())
print(df.shape)

                                              Tweets             User User_statuses_count user_followers      User_location User_verified fav_count rt_count           tweet_date                                        clean_tweet Sentiment
0  RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...  C. Anthony Null               41981           1479  Corinth, MS 38834         False         0        7  2021-06-23 06:18:19  RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...   Neutral
1  RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...       ABerman333               94023           1735                            False         0        7  2021-06-23 06:17:34  RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...   Neutral
2  RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATIO...         Mina A.S               16419            457             London         False         0        7  2021-06-23 06:16:44  RT TEXAS STATE BAR ASSOCIATION INVESTIGATING A...   Neutral
3  RT @BombshellDAILY: TEXAS STATE BAR ASSOC

# Picking up random tweets from 1000

In [21]:
n = 20
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
“They’re actually making it easier for unelected judges and partisan election boards to overturn the results of an… https://t.co/5LVJOwhXTO

Clean Tweet:
They re actually making it easier for unelected judges and partisan election boards to overturn the results of an

Sentiment:
Neutral


In [22]:
n = 202
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @CNN: Iranian President-elect Ebrahim Raisi ruled out a meeting with President Joe Biden and called on the US to return to the nuclear d…

Clean Tweet:
RT Iranian President elect Ebrahim Raisi ruled out a meeting with President Joe Biden and called on the US to return to the nuclear d

Sentiment:
Positive


In [23]:
n = 50
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @TheCB_News: Do you believe that Joe Biden fairly and legitimately won the 2020 presidential election? Let us get your thoughts on this…

Clean Tweet:
RT News Do you believe that Joe Biden fairly and legitimately won the 2020 presidential election Let us get your thoughts on this

Sentiment:
Positive


# Getting the sum of Positive, Negative, Neutral sentiments

In [24]:
df[df.Sentiment=='Positive'].shape[0]

572

In [25]:
df[df.Sentiment=='Neutral'].shape[0]

317

In [26]:
df[df.Sentiment=='Negative'].shape[0]

111

# Conclusion

# Data= "US election Joe Biden"
## Positive = 462
## Neutral = 308
## Negative = 230